## Colorspace

* https://paper.dropbox.com/doc/Colorspace-mZM1NPjjVfGrYZ4ztwUWC

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
p = os.path.join(os.path.dirname('__file__'), '..')
sys.path.append(p)
from common import *

## Data

In [ ]:
DATA_DIR = '../data/'
json_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.json')
img_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.png')

In [ ]:
%ls {DATA_DIR}
%ls {IMG_DIR}

## Helpers

In [ ]:
def plot_img(arr, fs=(10,10), cmap='gray', title=None):
    plt.figure(figsize=fs)
    plt.imshow(arr, cmap=cmap)
    plt.title(title)
    plt.show()
    
def load_img(fpath):
    return plt.imread(fpath)

def load_cv2_img(fpath, w=None, h=None, colorspace=None):
    img = cv2.imread(img_fpath)
    if colorspace is not None:
        img = cv2.cvtColor(img, colorspace)
    if None not in [w,h]:
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)
    return img

### BGR

In [ ]:
bgr_img = load_cv2_img(img_fpath)

# Shape (w,h,c)
print(bgr_img.shape)

# Plot
plot_img(bgr_img, fs=(10,10))

# Channels
b,g,r = bgr_img[:,:,0], bgr_img[:,:,1], bgr_img[:,:,2]

plot_img(b, title='Blue')
plot_img(g, title='Green')
plot_img(r, title='Red')

### RGB

* Additive - combines Red, Green, Blue values
* 3 Channels correlated by amount of light hitting surface
* Problems
    * Mixes color (chrominance) )and intensity (luminance) information into a single value

In [ ]:
rgb_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# Plot
plot_img(rgb_img, fs=(10,10))

### HSV

* Hue (Dominant wavelength)
* Saturation (Purity / shades of the color)
* Value (Intensity)

Pros
* Only one channel needed to describe color (H)
* Best for color thresholding (why?)
* More robust to reflections on the floor

Cons
* Device dependent

![HSV]](https://edoras.sdsu.edu/doc/matlab/toolbox/images/hsvcone.gif)

In [ ]:
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)

# Plot
plot_img(hsv_img, fs=(10,10))

# Channels (Hue, Saturation, Value)
h,s,v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]

plot_img(h, title='Hue')
plot_img(s, title='Saturation')
plot_img(v, title='Value')

In [ ]:
# Determine HSV value of specific color

blue = np.uint8([[[255,0,0 ]]])
hsv_blue = cv2.cvtColor(blue, cv2.COLOR_BGR2HSV)
print("Blue", hsv_blue)

green = np.uint8([[[0,255,0 ]]])
hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)
print("Green", hsv_green)

red = np.uint8([[[0,0,255]]])
hsv_red = cv2.cvtColor(red, cv2.COLOR_BGR2HSV)
print("Red", hsv_red)

## Find color of pixel

In [ ]:
def get_color_of_pixel(fpath, x, y, colorspace='BGR'):
    rgb_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
    hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
    
    rgb_colors = rgb_img[y,x,:]
    hsv_colors = hsv_img[y,x,:]
    bgr_colors = np.copy(rgb_colors[::-1])
    print ("BGR:", bgr_colors)
    print ("RGB:", rgb_colors)
    print ("HSV:", hsv_colors)
    
    # Plot to visualize
    img = np.copy(rgb_img)
    img[y-5:y+5:,x-5:x+5,:] = 255
    img[y,x,:] = 0
    plot_img(img, fs=(18,18))
    
    if colorspace == 'BGR':
        return bgr_colors.tolist()
    if colorspace == 'RGB':
        return rgb_colors.tolist()
    return hsv_colors.tolist()

get_color_of_pixel(img_fpath, 100, 100, 'RGB')

In [ ]:
def get_hsv_value_of_bgr(bgr_color):
    print("BGR", bgr_color)
    bgr_color = np.uint8([[bgr_color]])
    hsv = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]
    print("HSV", hsv)
    return hsv

blue = [255,0,0 ]
_ = get_hsv_value_of_bgr(blue)

## Color Histograms

In [ ]:
bgr_img = load_cv2_img(img_fpath)

# Cv2 Histogram (faster)
hist = cv2.calcHist(images=[bgr_img], channels=[0], mask=None, histSize=[256], ranges=[0,256])

# Numpy Histogram (slower)
hist, bins = np.histogram(bgr_img.ravel(), 256, [0,256])

In [ ]:
# Plotting Histogram (all channels flattened)
def plot_hist(img, bins=256, title=None):
    plt.hist(img.ravel(), bins=bins, range=[0,256])
    plt.title(title)
    plt.show()

plot_hist(bgr_img)

In [ ]:
def plot_bgr_hist(bgr_img, bins=256, mask=None):
    # Mask let's you select for certain regions    
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([bgr_img],[i],mask,[bins],[0,256])
        plt.plot(histr, color=col)
        plt.xlim([0,bins])
    plt.show()

plot_bgr_hist(bgr_img)

## Histograms

In [ ]:
img = cv2.imread(img_fpath)

In [ ]:
# Loading Histogram

# OpenCV - faster
hist = cv2.calcHist(images=[img], channels=[0], mask=None, histSize=[256], ranges=[0,256])

# Numpy
hist, bins = np.histogram(img.ravel(), 256, [0,256])

In [ ]:
# Plotting Histogram
plt.hist(img.ravel(), 256, [0,256])
plt.show()

In [ ]:
# Color histogram

def plot_color_hist(img, bins=256, mask=None):
    # Mask let's you select for certain regions
    
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([img],[i],mask,[bins],[0,256])
        plt.plot(histr, color=col)
        plt.xlim([0,bins])
    plt.show()

In [ ]:
## Applying a Mask

mask = np.zeros(img.shape[:2], np.uint8)
mask[400:700, 100:1100] = 255
masked_img = cv2.bitwise_and(img, img, mask=mask)

plot_img(load_cv2_img(img_fpath))
plot_img(mask)
plot_img(masked_img)

plot_color_hist(img, bins=30)
plot_color_hist(img, bins=30, mask=mask)

In [ ]:
# HSV Histogram - Hue holds the color information nicely
img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
hist = cv2.calcHist(images=[hsv], channels=[0], mask=None, histSize=[359], ranges=[0,359])
plt.plot(hist)

In [ ]:
hsv.shape

In [ ]:
# Color Quantization
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html

img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 6
ret, label, center = cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

plot_img(res2, fs=(12,8))

In [ ]:
plot_color_hist(res2)

## Links

* https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/
* https://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html